In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")


chat = ChatOpenAI(openai_api_key=api_key,
                  temperature=0.1,
                  streaming=True,
                  callbacks=[
                      StreamingStdOutCallbackHandler(),
                      ],)

examples = [
    {
        "country": "프랑스",
        "answer": """
        제가 아는 것은 다음과 같습니다:
        수도: 파리
        언어: 프랑스어
        음식: 와인과 치즈
        통화: 유로
        """,
    },
    {
        "country": "이탈리아",
        "answer": """
        제가 아는 것은 다음과 같습니다:
        수도: 로마
        언어: 이탈리아어
        음식: 피자와 파스타
        통화: 유로
        """,
    },
    {
        "country": "그리스",
        "answer": """
        제가 아는 것은 다음과 같습니다:
        수도: 아테네
        언어: 그리스어
        음식: 수블라키와 페타 치즈
        통화: 유로
        """,
    },
]



example_prompt = ChatPromptTemplate.from_messages([
    ("human",  "{country}에 대해 무엇을 알고 있나요?",),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 모든 나라를 새로운 컨텐츠를 분석하고 자세하게 알고있는 전문가입니다. "),
    example_prompt,
    ("human", "{country}에 대해 무엇을 알고 있나요?",)
    
])

chain = final_prompt | chat

chain.invoke({"country": "한국"})

한국에 대해 알고 있는 정보는 다음과 같습니다:
수도: 서울
언어: 한국어
음식: 김치, 불고기, 비빔밥
통화: 대한민국 원 (KRW)
문화: 한복, 태권도, 한류
경제: 세계적으로 경제력이 강한 나라 중 하나
특이사항: 한반도의 북쪽에는 북한이 위치해 있습니다.

AIMessageChunk(content='한국에 대해 알고 있는 정보는 다음과 같습니다:\n수도: 서울\n언어: 한국어\n음식: 김치, 불고기, 비빔밥\n통화: 대한민국 원 (KRW)\n문화: 한복, 태권도, 한류\n경제: 세계적으로 경제력이 강한 나라 중 하나\n특이사항: 한반도의 북쪽에는 북한이 위치해 있습니다.')